# **Install packages**

In [ ]:
# !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton
# !pip install --no-deps cut_cross_entropy unsloth_zoo
# !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
# !pip install --no-deps unsloth

# **Import necessary packages and libs**

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from transformers import TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from kaggle_secrets import UserSecretsClient

# **Prepare Dataset**

In [3]:
INPUT_PATH = "/kaggle/input/weather-conversations/weather-conversations.json"
OUTPUT_PATH = "/kaggle/working/weather_conversations_formatted.json"
SYSTEM_PROMPT = "You are a friendly and knowledgeable weather assistant. You explain weather phenomena clearly and accurately."

with open(INPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

formatted_data = []
for item in data:
    instruction = item.get("instruction", "").strip()
    response = item.get("response", "").strip()
    
    # LLaMA prompt format
    prompt_text = (
        f"<s>[INST] <<SYS>>\n"
        f"{SYSTEM_PROMPT}\n"
        f"<</SYS>>\n\n"
        f"{instruction} [/INST] {response} </s>"
    )
    
    formatted_data.append({"text": prompt_text})

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(formatted_data, f, indent=2, ensure_ascii=False)

# **GET APIs and Load Dataset**

In [ ]:
user_secrets = UserSecretsClient()
hugging_faces_token = user_secrets.get_secret("HF_Llama")

dataset = load_dataset("json",data_files="/kaggle/working/weather_conversations_formatted.json")

# **Visualize Dataset**

In [5]:
for i, sample in enumerate(dataset['train']):
    print(f'\n----- Sample {i + 1} -----')
    print(sample)
    if i > 2:
        break


----- Sample 1 -----
{'text': '<s>[INST] <<SYS>>\nYou are a friendly and knowledgeable weather assistant. You explain weather phenomena clearly and accurately.\n<</SYS>>\n\nCan you explain what is sleet? [/INST] Sleet is precipitation in the form of ice pellets, often mixed with rain or snow. </s>'}

----- Sample 2 -----
{'text': '<s>[INST] <<SYS>>\nYou are a friendly and knowledgeable weather assistant. You explain weather phenomena clearly and accurately.\n<</SYS>>\n\nWhat does it mean when we say what is frost? [/INST] Frost occurs when water vapor freezes on surfaces when the temperature drops below 0°C. </s>'}

----- Sample 3 -----
{'text': '<s>[INST] <<SYS>>\nYou are a friendly and knowledgeable weather assistant. You explain weather phenomena clearly and accurately.\n<</SYS>>\n\nCan you explain what causes rain? [/INST] Rain occurs when water vapor in clouds condenses into droplets that become heavy enough to fall to the ground. </s>'}

----- Sample 4 -----
{'text': '<s>[INST] 

# **Load model**

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = False,
    token=hugging_faces_token
)

In [7]:
tokenizer.clean_up_tokenization_spaces = False

# **LoRA Configs**

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2025.10.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# **Fine-tuning**

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=5,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"
    )
)

In [10]:
# trainer_stats = trainer.train()